<div align="center">

# **Limpieza**

</div>

## Librerias

In [303]:
library(dplyr)
library(tidyr)
source("../src/data/dividir_polizas_por_anio.R")
source("../src/data/ajustar_inflacion.R")
source("../src/data/Freedman_Diaconis.R")

## Data

In [304]:
df_input <- read.csv("../data/input/Muestra_Siniestros_4.csv")
dim(df_input)
glimpse(df_input)

[1] 5000   18

Rows: 5,000
Columns: 18
$ Amparo          <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, "PERDI…
$ Amp             <chr> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, "PPD",…
$ SumaDePagos     <dbl> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, 263295…
$ Modelo          <int> 2013, 2010, 2012, 2008, 2008, 2007, 2012, 2007, 2010, …
$ Color           <chr> "ROJO", "PLATA", "PLATA", "BLANCO", "VINO TINTO", "PLA…
$ Carroceria      <chr> "SEDAN", "HATCHBACK", "SEDAN", "SEDAN", "SEDAN", "HATC…
$ MARCA           <chr> "HYUNDAI", "HYUNDAI", "HYUNDAI", "HYUNDAI", "HYUNDAI",…
$ Referencia1     <chr> "VELOSTER", "ATOS [2]", "ACCENT i25", "ATOS [2]", "ACC…
$ Referencia2     <chr> "COUPE", "PRIME", "1.4L", "PRIME", "GLS", "SANTRO", "G…
$ CLASE_FASECOLDA <chr> "AUTOMOVIL", "AUTOMOVIL", "AUTOMOVIL", "AUTOMOVIL TAXI…
$ TIPO_VEHICULO   <chr> "Livianos", "Livianos", "Livianos", "Livianos", "Livia…
$ SERVICIO        <chr> "Particular", "Particular", "Particular", "Particular"…
$ Sexo_Aseg     

In [305]:
unique(df_input$MARCA)

[1] "HYUNDAI"

In [306]:
nrow(df_input)

[1] 5000

In [307]:
colSums(is.na(df_input))

Amparo             Amp     SumaDePagos          Modelo           Color 
           4320            4320            4320               0               0 
     Carroceria           MARCA     Referencia1     Referencia2 CLASE_FASECOLDA 
              0               0               0               0               0 
  TIPO_VEHICULO        SERVICIO       Sexo_Aseg            Edad           Desde 
              0               0             625               0               0 
          Hasta    Vr_Comercial            Pago 
              0               0               0

## Preparacion de los datos

Nombres de variables

In [308]:
df_input$Color <- gsub("CHAMPA�A", "CHAMPAÑA", df_input$Color)
df_input$Amparo <- gsub("DA�OS", "DAÑOS", df_input$Amparo)

Nulos en Amp

In [309]:
df_input <- df_input %>%
  mutate(
    # Convertir NA a "No aplica"
    Amparo = ifelse(is.na(Amparo), "No aplica", Amparo),
    Amp = ifelse(is.na(Amp), "No aplica", Amp)
    )

Nulos en SumaDePagos

In [310]:
head(sort(unique(df_input$SumaDePagos)))

[1]     0 26248 60000 69257 70000 80000

In [311]:
pago_minimo <- 100000
df_input <- df_input %>% mutate(
    SumaDePagos = case_when(
      is.na(SumaDePagos) ~ 0,
      SumaDePagos < pago_minimo ~ 0,
      TRUE ~ SumaDePagos)
)

df_input <- df_input %>% mutate(
  Accidentado = ifelse(SumaDePagos >= pago_minimo, 1, 0)
)

El pago minimo se escogio como el valor de un espejo en 2015

Nulos en genero

In [312]:
df_input <- df_input %>% mutate(
    Sexo_Aseg = ifelse(is.na(Sexo_Aseg), "No aplica", Sexo_Aseg)
)

In [313]:
nrow(df_input[(df_input$Sexo_Aseg == "No aplica"), ])/nrow(df_input)

[1] 0.125

In [314]:
df_input <- df_input[(df_input$Sexo_Aseg != "No aplica"), ]

Segmentando por año

In [315]:
df_input <- dividir_polizas_por_anio(df_input)

Calculo de exposicion

In [316]:
df_input$exposicion <- as.numeric(df_input$Hasta - df_input$Desde) + 1

Ajustando a inflacion de 2015

In [317]:
max(df_input$Hasta)

[1] "2015-02-21"

In [318]:
df_input <- ajustar_inflacion_2015(df_input)

Llevamos todo a los mismo valores presentes ajustando la inflacion.

Variables que sobran

In [319]:
df_input[c("Amp", "MARCA")] <- NULL

Ajustando valor comercial

In [320]:
vr_minimo <- 4000000
df_input <- df_input[(df_input$Vr_Comercial > vr_minimo), ]

El valor comercial minimo se escogio como el valor de un carro hyundai Excel en 2015

Ajustando edad

In [321]:
edad_minima <- 18
edad_maxima <- 90

df_input <- df_input[(df_input$Edad >= edad_minima & df_input$Edad <= edad_maxima), ]

In [322]:
df_input$mes <- as.numeric(format(as.Date(df_input$Hasta), "%m"))

df_input$Desde <- NULL
df_input$Hasta <- NULL

In [323]:
print("Columnas disponibles:")
print(colnames(df_input))
print("¿Existe la columna Edad?")
print("Edad" %in% colnames(df_input))
if("Edad" %in% colnames(df_input)) {
  print("Tipo de datos de Edad:")
  print(class(df_input$Edad))
}

[1] "Columnas disponibles:"
 [1] "Amparo"          "SumaDePagos"     "Modelo"          "Color"          
 [5] "Carroceria"      "Referencia1"     "Referencia2"     "CLASE_FASECOLDA"
 [9] "TIPO_VEHICULO"   "SERVICIO"        "Sexo_Aseg"       "Edad"           
[13] "Vr_Comercial"    "Pago"            "Accidentado"     "exposicion"     
[17] "mes"            
[1] "¿Existe la columna Edad?"
[1] TRUE
[1] "Tipo de datos de Edad:"
[1] "integer"


In [324]:
if("Edad" %in% colnames(df_input)) {
  df_input$Edad <- as.numeric(df_input$Edad)
  print("Conversión exitosa. Edad es ahora:")
  print(class(df_input$Edad))
  print("Resumen de la columna Edad:")
  print(summary(df_input$Edad))
} else {
  print("ERROR: La columna Edad no existe en el dataframe.")
  print("Parece que ya fue eliminada en algún punto anterior.")
}

[1] "Conversión exitosa. Edad es ahora:"
[1] "numeric"
[1] "Resumen de la columna Edad:"
   Min. 1st Qu.  Median    Mean 3rd Qu.    Max. 
  19.00   36.00   44.00   45.46   53.00   85.00 


In [325]:
if("Edad" %in% colnames(df_input)) {
  resultado_fd <- Freedman_Diaconis(df_input$Edad, n_categories = 3)

  print("Resumen de categorías:")
  print(resultado_fd$summary)

  print("Puntos de corte:")
  print(resultado_fd$breaks)

  print("Nombres de las categorías:")
  print(resultado_fd$category_labels)

  for (i in 1:length(resultado_fd$category_labels)) {
    category_name <- resultado_fd$category_labels[i]
    df_input[[category_name]] <- as.numeric(resultado_fd$categories == category_name)
  }

  df_input$Edad <- NULL
  print("Segmentación de edad completada y columna Edad eliminada.")
} else {
  print("No se puede aplicar Freedman-Diaconis: la columna Edad no existe.")
  print("Verifique si ya fue procesada anteriormente.")
}

[1] "Resumen de categorías:"
categories
Edad_19_41 Edad_41_63 Edad_63_85 
      2728       3448        574 
[1] "Puntos de corte:"
[1] 18.999 41.000 63.000 85.001
[1] "Nombres de las categorías:"
[1] "Edad_19_41" "Edad_41_63" "Edad_63_85"
[1] "Segmentación de edad completada y columna Edad eliminada."


Aplicando Freedman-Diaconis para segmentar edad

Resultados

In [326]:
colSums(is.na(df_input))

Amparo     SumaDePagos          Modelo           Color      Carroceria 
              0               0               0               0               0 
    Referencia1     Referencia2 CLASE_FASECOLDA   TIPO_VEHICULO        SERVICIO 
              0               0               0               0               0 
      Sexo_Aseg    Vr_Comercial            Pago     Accidentado      exposicion 
              0               0               0               0               0 
            mes      Edad_19_41      Edad_41_63      Edad_63_85 
              0               0               0               0

In [327]:
summary(df_input)

    Amparo           SumaDePagos           Modelo        Color          
 Length:6750        Min.   :       0   Min.   :1993   Length:6750       
 Class :character   1st Qu.:       0   1st Qu.:2008   Class :character  
 Mode  :character   Median :       0   Median :2010   Mode  :character  
                    Mean   :  281823   Mean   :2009                     
                    3rd Qu.:       0   3rd Qu.:2012                     
                    Max.   :35206209   Max.   :2013                     
  Carroceria        Referencia1        Referencia2        CLASE_FASECOLDA   
 Length:6750        Length:6750        Length:6750        Length:6750       
 Class :character   Class :character   Class :character   Class :character  
 Mode  :character   Mode  :character   Mode  :character   Mode  :character  
                                                                            
                                                                            
                           

Exportando los resultados

In [328]:
write.csv(df_input, "../data/processed/datos_limpios.csv", row.names = FALSE)